In [9]:
import pandas as pd
import calendar

#Calculate the efficiency score
def calculate_efficiency_score_row(row):
    if row['Duration'] < row['Expected Duration']:
        return float(format((row['Expected Duration'] - row['Duration']) / row['Expected Duration'], '.3f'))
    else:
        return float(format(((row['Duration'] - row['Expected Duration']) / row['Expected Duration'] * (-1)), '.3f'))
    
def calculate_performance_score(efficiency, rating, success_rate):

    performance_score = format(((efficiency * 100 * 0.4 * 2) + (rating * 20 * 0.3) + (success_rate * 100 * 0.3)), '.3f')

    return float(performance_score)

def performance_score_sheet(dummy_data, efficiency_data):

    dummy_data['Rating'] = pd.to_numeric(dummy_data['Rating'], errors='coerce')
    dummy_data['Success'] = pd.to_numeric(dummy_data['Success'], errors='coerce')

    grouped_data = dummy_data.groupby(['Dentist Name', 'Treatment']).agg({
        'Rating': 'mean',
        'Success': 'mean'
    }).reset_index()

    grouped_data = grouped_data.merge(efficiency_data.melt(id_vars='Dentist Name', var_name='Treatment', value_name='Efficiency'), on=['Dentist Name', 'Treatment'], how='left')

    grouped_data['Performance Score'] = grouped_data.apply(
        lambda row: calculate_performance_score(row['Efficiency'], row['Rating'], row['Success']), axis=1)

    performance_df = grouped_data.pivot(index='Dentist Name', columns='Treatment', values='Performance Score').reset_index()
    performance_df = performance_df.fillna(0)

    return performance_df



def get_performance_metrics(dentist_name, treatment, df, df2):
    target_dentist = df[df['Dentist Name'] == dentist_name]
    target_treatment = target_dentist[target_dentist['Treatment'] == treatment]

    target_success = target_treatment[target_treatment['Success'] == 1]

    target_expected_duration = df2[df2['Treatment Menu Names'] == treatment]
    target_expected_duration = target_expected_duration['Duration'].values[0]

    count = target_treatment.value_counts().sum()
    average = format(target_treatment['Duration'].mean(), '.3f')
    max = format(target_treatment['Duration'].max(), '.3f')
    min = format(target_treatment['Duration'].min(), '.3f')
    std = format(target_treatment['Duration'].std(), '.3f')
    rating_avg = float(format(target_treatment['Rating'].mean(), '.1f'))
    success_rate = float(target_success['Success'].value_counts().sum())/float(count)

   # Calculate the performance score
    if float(average) < float(target_expected_duration):
        efficiency = (target_expected_duration - float(average)) / target_expected_duration * 1
    else:
        efficiency = (float(average) - target_expected_duration) / target_expected_duration * -1

    efficiency = float(format(efficiency, '.3f'))

    performance_score = calculate_performance_score(efficiency, rating_avg, success_rate)


    return(count, average, max, min, std, efficiency, rating_avg, success_rate, float(performance_score))


def get_performance_metrics_chart(dentist_name, treatment, df, df2):
    target_dentist = df[df['Dentist Name'] == dentist_name]
    target_treatment = target_dentist[target_dentist['Treatment'] == treatment]

    target_expected_duration = df2[df2['Treatment Menu Names'] == treatment]
    target_expected_duration = target_expected_duration['Duration'].values[0]

    # Extract the first and last dates
    start_date = df['Date'].min()
    end_date = df['Date'].max()

    # Convert month numbers to month names
    start_month = calendar.month_name[start_date.month]
    end_month = calendar.month_name[end_date.month]

    # Extract the series for plotting
    treatment_duration_series = target_treatment[['Date', 'Duration']].set_index('Date')

    return(treatment_duration_series, target_expected_duration, start_month, end_month)


def Anlyze_dentist_past_data(df_dentist_report, df_service):

    dentists = df_dentist_report['Dentist Name'].unique()

    # Ensure 'Duration' column is numeric
    df_dentist_report['Duration'] = pd.to_numeric(df_dentist_report['Duration'])

    # Calculate the average duration for each treatment per dentist
    average_duration_per_dentist_treatment = df_dentist_report.groupby(['Dentist Name', 'Treatment']).agg({'Duration': 'mean'}).reset_index()

    # Merge with the expected durations from the treatment data
    expected_durations = df_service.rename(columns={'Treatment Menu Names': 'Treatment', 'Duration': 'Expected Duration'})
    efficiency_data = average_duration_per_dentist_treatment.merge(expected_durations, on='Treatment')

    efficiency_data['Efficiency Score'] = efficiency_data.apply(calculate_efficiency_score_row, axis=1)


    # Pivot the data to get the desired format for average duration
    average_duration_df = average_duration_per_dentist_treatment.pivot(index='Dentist Name', columns='Treatment', values='Duration').reset_index()

    # Create payload1 in JSON format
    payload1 = {
        "analyzed_sheet": "average_durations",
        "df_content": []
    }

    for _, row in average_duration_df.iterrows():
        dentist_entry = {
            "dentist_name": row['Dentist Name']
        }
        for treatment in average_duration_df.columns[1:]:  # Skipping the 'Dentist Name' column
            dentist_entry[treatment] = row[treatment]
        payload1["df_content"].append(dentist_entry)


    # Pivot the data to get the desired format for performance evaluation
    evaluation_df = efficiency_data.pivot(index='Dentist Name', columns='Treatment', values='Efficiency Score').reset_index()

    # Create payload2 in JSON format
    payload2 = {
        "analyzed_sheet": "efficiency_scores",
        "df_content": []
    }

    for _, row in evaluation_df.iterrows():
        dentist_entry = {
            "dentist_name": row['Dentist Name']
        }
        for treatment in evaluation_df.columns[1:]:  # Skipping the 'Dentist Name' column
            dentist_entry[treatment] = row[treatment]
        payload2["df_content"].append(dentist_entry)


    # Calculate treatment score for each dentist
    performance_score_df = performance_score_sheet(df_dentist_report, evaluation_df)

    payload3 = {
        "analyzed_sheet": "performance_scores",
        "df_content": []
    }

    for _, row in performance_score_df.iterrows():
        dentist_entry = {
            "dentist_name": row['Dentist Name']
        }
        for treatment in performance_score_df.columns[1:]:  # Skipping the 'Dentist Name' column
            dentist_entry[treatment] = row[treatment]
        payload3["df_content"].append(dentist_entry)

   
    # Create the Specialty DataFrame
    specialty_data = []

    # Create the Specialty DataFrame for payload4
    payload4 = {
        "analyzed_sheet": "specialties",
        "df_content": []
    }

    for dentist in dentists:
        specialty_treatments = efficiency_data[(efficiency_data['Dentist Name'] == dentist) & (efficiency_data['Efficiency Score'] > 0.02)]
        specialties = specialty_treatments['Treatment'].tolist()
        efficiency_scores = specialty_treatments['Efficiency Score'].tolist()
        specialty_data = {
            'Dentist Name': dentist,
            'Specialty': specialties,
            'Efficiency': efficiency_scores,
            'Wage/Hour': "Edit This!"
        }
        payload4["df_content"].append(specialty_data)


    return [payload1, payload2, payload3, payload4]


df_dentist_report = pd.read_excel("./Data/Dummy_Treatment.xlsx", sheet_name="Dummy Treatments")
df_service = pd.read_excel("./Data/Service_list.xlsx", sheet_name="Main")

print(Anlyze_dentist_past_data(df_dentist_report, df_service))

({'analyzed_sheet': 'average_durations', 'df_content': [{'dentist_name': 'Arnold', 'Additional Root': 48.884434845224625, 'Advanced Filling': 24.323328302248214, 'Advanced Gum Treatment': 48.523827983882654, 'Basic Filling': 51.59288958829579, 'Basic Gum Treatment': 35.335104265758694, 'Basic Scaling': 33.694480618823285, 'Basic Tooth Extraction': 31.64343293380553, 'Basic X-ray': 3.9628158295775444, 'Consultation': 25.3135680484807, 'Dental Post & Core': 71.97060442764442, 'Dental Spa': 14.43032446965361, 'Membership Consultation Perk': 31.72642413725763, 'Mouth Guard': 36.18248189041754, 'Premium Bridge': 71.40330431321196, 'Premium Crown': 72.4197199420494, 'Prevention Seal': 16.115378415761203, 'Root Canal Treatment': 72.36980393891525, 'Wear Protect': 35.68123468639256}, {'dentist_name': 'Ashley', 'Additional Root': 59.66964134667357, 'Advanced Filling': 29.703913409878016, 'Advanced Gum Treatment': 58.90591767201521, 'Basic Filling': 66.17383284859814, 'Basic Gum Treatment': 44.9

## Run Sheet Scheduling V1 

In [1]:
import pandas as pd
import numpy as np

TaskClassification_directory = "./Data/TaskClassification.xlsx"

capabilities_df = pd.read_excel(TaskClassification_directory, sheet_name='Capabilities')
constraints_df = pd.read_excel(TaskClassification_directory, sheet_name='Constraints')
worker_df = pd.read_excel(TaskClassification_directory, sheet_name='Worker')
scenario_df = pd.read_excel("./data/forecast_2_weeks.xlsx")
scenario_df = scenario_df.drop(columns='Hour Required', axis=1)
service_df = pd.read_excel("./data/Service_list.xlsx")
service_df = service_df[['Treatment Menu Names', 'Duration']]
service_df.columns = ['Treatment', 'Duration']

# Define the clinic conditions based on constraints
num_rooms = int(constraints_df.loc[constraints_df['Constraints'] == 'Room_num', 'Value'].values[0])
treatment_interval = int(constraints_df.loc[constraints_df['Constraints'] == 'Treatment_Interval', 'Value'].values[0])
clinic_open_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Open', 'Value'].values[0]
clinic_close_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Close', 'Value'].values[0]
max_working_hour_day = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Day', 'Value'].values[0]) * 60  # Convert hours to minutes
max_working_hour_week = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Week', 'Value'].values[0]) * 60  # Convert hours to minutes

# Room Configurations
chairs_in_rooms = {
    'Room1': ['Chair1', 'Chair2'],
    'Room2': ['Chair1', 'Chair2'],
    'Room3': ['Chair3']  # Room3 is for advanced treatments, always uses Chair3
}

# Determine advanced treatments: those that Chair3 can do but Chair2 cannot
chair3_treatments = set(capabilities_df.columns[1:][capabilities_df.loc[capabilities_df['Object-Subject'] == 'Chair3'].iloc[0, 1:].astype(bool)])
chair2_treatments = set(capabilities_df.columns[1:][capabilities_df.loc[capabilities_df['Object-Subject'] == 'Chair2'].iloc[0, 1:].astype(bool)])
chair1_treatments = set(capabilities_df.columns[1:][capabilities_df.loc[capabilities_df['Object-Subject'] == 'Chair1'].iloc[0, 1:].astype(bool)])
advanced_treatments = chair3_treatments - chair2_treatments

# Reshape the forecast_2_weeks DataFrame to have a 'Treatment' and 'Count' for each date
forecast_reshaped = scenario_df.melt(id_vars=['Date'], 
                                     var_name='Treatment', 
                                     value_name='Count')

# Filter out the 'Hour Required' from the forecast data
forecast_reshaped = forecast_reshaped[forecast_reshaped['Treatment'] != 'Hour Required']

# Filter out treatments with a count of 0
forecast_reshaped = forecast_reshaped[forecast_reshaped['Count'] > 0]

# Map worker types to their hourly wages and IDs
worker_wages = worker_df.set_index('Worker_Id')['Wage_Hour'].to_dict()
job_to_worker_ids = worker_df.groupby('Job')['Worker_Id'].apply(list).to_dict()

# Prepare to track worker hours
worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_last_end_time = {worker_id: pd.to_datetime(forecast_reshaped['Date'].min()) for worker_id in worker_df['Worker_Id']}

# Update the treatment_worker_map using Worker_Id based on Job
treatment_worker_map = {}
for treatment in capabilities_df.columns[1:]:
    eligible_jobs = capabilities_df.loc[capabilities_df[treatment] == True, 'Object-Subject']
    available_workers = [worker_id for job in eligible_jobs for worker_id in job_to_worker_ids.get(job, [])]
    treatment_worker_map[treatment] = available_workers

# Prepare to store the scheduled output
output_schedule = {room: [] for room in chairs_in_rooms.keys()}

# Initialize time tracking for each room
current_time = {room: pd.to_datetime(f"{forecast_reshaped['Date'].min()} {clinic_open_time}") for room in chairs_in_rooms.keys()}

# Keep track of which room to assign the next non-advanced treatment
non_advanced_room_toggle = True

# Track the current week to reset weekly hours
current_week = pd.to_datetime(forecast_reshaped['Date'].min()).isocalendar()[1]

# Sort the dates to ensure they are processed in chronological order
sorted_dates = sorted(forecast_reshaped['Date'].unique())

# Iterate over each unique date in sorted order
for date in sorted_dates:
    print(f"Processing date: {date}")
    
    # Reset weekly hours if it's a new week
    week_of_year = pd.to_datetime(date).isocalendar()[1]
    if week_of_year != current_week:
        worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
        current_week = week_of_year
    
    # Reset daily hours for workers
    worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
    
    # Reset or update last end time appropriately for new day
    for worker_id in worker_last_end_time:
        worker_last_end_time[worker_id] = pd.to_datetime(date).replace(hour=0, minute=0, second=0)
    
    # Reset current time for each room to the clinic's opening time
    current_time = {room: pd.to_datetime(f"{date} {clinic_open_time}") for room in chairs_in_rooms.keys()}
    
    # Diagnostic output to ensure reset is correct
    print(f"Worker hours reset for {date}:")
    print(f"Daily hours: {worker_hours_daily}")
    print(f"Weekly hours: {worker_hours_weekly}")
    print(f"Worker last end time: {worker_last_end_time}")
    print(f"Current time for rooms: {current_time}")
    
    # Continue with the scheduling logic
    daily_treatments = forecast_reshaped[forecast_reshaped['Date'] == date]


    # Iterate over each treatment for the day
    for index, row in daily_treatments.iterrows():
        treatment = row['Treatment']
        count = int(row['Count'])  # Number of sessions for this treatment
        
        # Get the treatment duration from the service list and introduce variability
        base_duration = service_df.loc[service_df['Treatment'] == treatment, 'Duration'].values[0]
        
        # Determine the room where the treatment is to be scheduled
        if treatment in advanced_treatments:
            room = 'Room3'
        else:
            room = 'Room1' if non_advanced_room_toggle else 'Room2'
            non_advanced_room_toggle = not non_advanced_room_toggle
        
        for i in range(count):  # Schedule each instance of the treatment sequentially
            eligible_workers = treatment_worker_map[treatment]
            selected_worker = None

            variation_factor = np.random.uniform(0.8, 1.2)  # ±20% variability
            treatment_duration = int(base_duration * variation_factor)
            
            for worker_id in eligible_workers:
                daily_available = max_working_hour_day - worker_hours_daily[worker_id]
                weekly_available = max_working_hour_week - worker_hours_weekly[worker_id]
                start_time = current_time[room]  # Start time is specific to the room
                end_time = start_time + pd.to_timedelta(treatment_duration, unit='m')
                
                # Debugging of Proposed Worker to satisfy the demands
                print(f"Worker: {worker_id}, daily: {daily_available}, Weekly: {weekly_available}, Proposed start time: {start_time}, Proposed end time: {end_time}, Treatment: {treatment}, Room: {room}")
                
                if (daily_available >= treatment_duration and
                    weekly_available >= treatment_duration and
                    worker_last_end_time[worker_id] <= start_time):
                    selected_worker = worker_id
                    worker_last_end_time[worker_id] = end_time
                    break

            if selected_worker is None:
                print(f"No worker could be selected for {treatment} on {date}. Rechecking all eligible workers.")
                for worker_id in eligible_workers:
                    start_time = current_time[room]
                    end_time = start_time + pd.to_timedelta(treatment_duration, unit='m')
                    if (worker_hours_daily[worker_id] < max_working_hour_day and
                        worker_hours_weekly[worker_id] < max_working_hour_week and
                        worker_last_end_time[worker_id] <= start_time):
                        selected_worker = worker_id
                        worker_last_end_time[worker_id] = end_time
                        break
            
            if selected_worker is None:
                raise ValueError(f"No available worker for treatment {treatment} on {date}")
        
            
            # Schedule the treatment
            output_schedule[room].append({
                'Date': pd.to_datetime(date).date(),
                'Start_Time': start_time.strftime('%I:%M %p'),
                'Finish_Time': end_time.strftime('%I:%M %p'),
                'Treatment': treatment,
                'Duration': treatment_duration,
                'Worker_Id': selected_worker,
                'Chair': 'Chair3' if room == 'Room3' else 'Chair1' if treatment in chair1_treatments else 'Chair2',
                'LaborCost': treatment_duration * (worker_wages[selected_worker] / 60)
            })
            
            # Update the current time for the room
            current_time[room] += pd.to_timedelta(treatment_duration + treatment_interval, unit='m')
            
            # Update worker hours
            worker_hours_daily[selected_worker] += treatment_duration
            worker_hours_weekly[selected_worker] += treatment_duration
            
            # Check if we exceed the clinic's operating hours for this room
            if current_time[room].time() > clinic_close_time:
                break
            
# Convert the schedule dictionary to DataFrames for each room
room_schedules = {room: pd.DataFrame(schedule) for room, schedule in output_schedule.items()}

# Add Total_Cost column for each room
for room, schedule in room_schedules.items():
    total_cost = schedule['LaborCost'].sum()
    # Add Total_Cost column with the total cost in the first row and the remaining rows empty
    schedule['Total_Cost'] = ''
    schedule.loc[0, 'Total_Cost'] = total_cost
    room_schedules[room] = schedule

# Save the output to an Excel file with separate sheets for each room
output_file_path = 'clinic_scheduleV1.xlsx'
with pd.ExcelWriter(output_file_path) as writer:
    for room, schedule in room_schedules.items():
        schedule.to_excel(writer, sheet_name=room, index=False)



Processing date: 2023-07-03
Worker hours reset for 2023-07-03:
Daily hours: {'N1': 0, 'N2': 0, 'OT1': 0, 'OT2': 0, 'D1N1': 0, 'D1N2': 0, 'D2N1': 0, 'D3N1': 0}
Weekly hours: {'N1': 0, 'N2': 0, 'OT1': 0, 'OT2': 0, 'D1N1': 0, 'D1N2': 0, 'D2N1': 0, 'D3N1': 0}
Worker last end time: {'N1': Timestamp('2023-07-03 00:00:00'), 'N2': Timestamp('2023-07-03 00:00:00'), 'OT1': Timestamp('2023-07-03 00:00:00'), 'OT2': Timestamp('2023-07-03 00:00:00'), 'D1N1': Timestamp('2023-07-03 00:00:00'), 'D1N2': Timestamp('2023-07-03 00:00:00'), 'D2N1': Timestamp('2023-07-03 00:00:00'), 'D3N1': Timestamp('2023-07-03 00:00:00')}
Current time for rooms: {'Room1': Timestamp('2023-07-03 08:00:00'), 'Room2': Timestamp('2023-07-03 08:00:00'), 'Room3': Timestamp('2023-07-03 08:00:00')}
Worker: OT1, daily: 480, Weekly: 2400, Proposed start time: 2023-07-03 08:00:00, Proposed end time: 2023-07-03 08:43:00, Treatment: Basic Scaling, Room: Room1
Worker: D1N1, daily: 480, Weekly: 2400, Proposed start time: 2023-07-03 08:00:

## Treatment item id JSON

In [61]:
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

with open("./treatment_precedence.json", "r") as file:
    item_numbers_data = json.load(file)

# Determine which worker do which treatment
def get_worker_treatments(worker_code, json_data):
    worker_treatments_list = []

    for treatment in json_data['treatment']:
        for detail in treatment['treatment_details']:
            for variation in detail['variation']:
                if variation['variation_worker_responsibility'] == worker_code:
                    worker_treatments_list.append(str(detail['item_number']) + "-" + variation['variation_code'])

    return worker_treatments_list

worker_treatments = {
    "OHT" : get_worker_treatments("OHT", item_numbers_data),
    "LV1" : get_worker_treatments("LV1", item_numbers_data),
    "LV2" : get_worker_treatments("LV2", item_numbers_data),
    "LV3" : get_worker_treatments("LV3", item_numbers_data)
}

print(worker_treatments)


# Extract all unique treatment codes
unique_treatment_codes = set()

for treatments in worker_treatments.values():
    unique_treatment_codes.update(treatments)

# Convert the set to a list if needed
unique_treatment_codes_list = list(unique_treatment_codes)

# Function to sort treatment codes
def sort_treatment_code(treatment_code):
    # Split the treatment code into numeric and alphabetic parts
    match = re.match(r"(\d+)-([A-Z]+)", treatment_code)
    if match:
        number_part = int(match.group(1))  # Convert the numeric part to an integer
        letter_part = match.group(2)       # Get the alphabetic part
        return (number_part, letter_part)
    return (treatment_code, "")

# Sort the treatment codes
unique_treatment_codes_list.sort(key=sort_treatment_code)

print(unique_treatment_codes_list)

scenario_df = pd.DataFrame(columns=unique_treatment_codes_list)

start_date = datetime.today()
scenario_df['Date'] = [start_date + timedelta(days = i) for i in range(14)]

for item_number in unique_treatment_codes_list:
    scenario_df[item_number] = np.random.randint(0, 3, size=len(scenario_df))

print(f"sum of treatment counts in the first day: {scenario_df.loc[0, unique_treatment_codes_list].sum()}")
print(scenario_df.head())

scenario_df.to_csv("./Scenario_itemNumber2.csv", index=False)


{'OHT': ['1001-A', '1001-C', '1002-A', '1003-A', '1003-B', '1004-A', '1004-B', '1004-C', '1004-E', '1005-A', '1005-B', '1005-C', '3002-A', '3002-B', '3003-A', '3003-B', '3003-C', '3004-A', '3005-A', '4007-A', '4008-A', '4009-A'], 'LV1': ['1001-B', '1004-D', '3014-A'], 'LV2': ['3005-B', '3015-A', '4001-A', '4002-A', '4003-A', '4005-A', '4006-A', '5007-A', '5007-B', '5008-A', '5009-A'], 'LV3': ['3005-C', '3005-D', '3006-A', '3007-A', '3008-A', '3009-A', '3010-A', '3011-A', '3012-A', '3013-A', '4004-A', '5001-A', '5002-A', '5003-A', '5004-A', '5005-A', '5006-A']}
['1001-A', '1001-B', '1001-C', '1002-A', '1003-A', '1003-B', '1004-A', '1004-B', '1004-C', '1004-D', '1004-E', '1005-A', '1005-B', '1005-C', '3002-A', '3002-B', '3003-A', '3003-B', '3003-C', '3004-A', '3005-A', '3005-B', '3005-C', '3005-D', '3006-A', '3007-A', '3008-A', '3009-A', '3010-A', '3011-A', '3012-A', '3013-A', '3014-A', '3015-A', '4001-A', '4002-A', '4003-A', '4004-A', '4005-A', '4006-A', '4007-A', '4008-A', '4009-A', '5

## Run sheet scheduling V2

In [23]:
import json
import pandas as pd
import re

#Load data
TaskClassification_directory_v2 = "./Data/TaskClassificationV2.xlsx"
scenario_df_v2 = pd.read_csv("./Scenario_itemNumber2.csv")
constraints_df = pd.read_excel(TaskClassification_directory_v2, sheet_name='Constraints')
worker_df = pd.read_excel(TaskClassification_directory_v2, sheet_name='Worker')
with open("./treatment_precedence.json", "r") as file:
    item_numbers_data = json.load(file)

# Define the clinic conditions based on constraints
num_rooms = int(constraints_df.loc[constraints_df['Constraints'] == 'Room_num', 'Value'].values[0])
treatment_interval = int(constraints_df.loc[constraints_df['Constraints'] == 'Treatment_Interval', 'Value'].values[0])
clinic_open_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Open', 'Value'].values[0]
clinic_close_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Close', 'Value'].values[0]
max_working_hour_day = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Day', 'Value'].values[0]) * 60  # Convert hours to minutes
max_working_hour_week = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Week', 'Value'].values[0]) * 60  # Convert hours to minutes


# Room Configurations
chairs_in_rooms = {
    'Remote': ['Chair0'],
    'Room1': ['Chair1', 'Chair2'],
    'Room2': ['Chair1', 'Chair2'],
    'Room3': ['Chair3']  # Room3 is for advanced treatments, always uses Chair3
}

# Determine each chair treatments
def get_chair_treatments(chair_number, json_data):

    chair_treatments_list = []

    for treatment in json_data['treatment']:
        for detail in treatment['treatment_details']:
            for variation in detail['variation']:
                if variation['chair'] == chair_number:
                    chair_treatments_list.append(str(detail['item_number']) + "-" + variation['variation_code'])

    return chair_treatments_list
                
chair1_treatments = get_chair_treatments(1, item_numbers_data)
chair2_treatments = get_chair_treatments(2, item_numbers_data)
chair3_treatments = get_chair_treatments(3, item_numbers_data)
chair0_treatments = get_chair_treatments(0, item_numbers_data)

# Determine which worker do which treatment
def get_worker_treatments(worker_code, json_data):
    worker_treatments_list = []

    for treatment in json_data['treatment']:
        for detail in treatment['treatment_details']:
            for variation in detail['variation']:
                if variation['variation_worker_responsibility'] == worker_code:
                    worker_treatments_list.append(str(detail['item_number']) + "-" + variation['variation_code'])

    return worker_treatments_list

worker_treatments = {
    "OHT" : get_worker_treatments("OHT", item_numbers_data),
    "LV1" : get_worker_treatments("LV1", item_numbers_data),
    "LV2" : get_worker_treatments("LV2", item_numbers_data),
    "LV3" : get_worker_treatments("LV3", item_numbers_data)
}
# Reshape the forecast_2_weeks DataFrame to have a 'Treatment' and 'Count' for each date
forecast_reshaped = scenario_df_v2.melt(id_vars=['Date'], 
                                     var_name='Treatment', 
                                     value_name='Count')

# Filter out treatments with a count of 0
forecast_reshaped = forecast_reshaped[forecast_reshaped['Count'] > 0]


# Map worker types to their hourly wages and IDs
worker_wages = worker_df.set_index('Worker_Id')['Wage_Hour'].to_dict()
job_to_worker_ids = worker_df.groupby('Job')['Worker_Id'].apply(list).to_dict() #Change the "JOB" Column format later 

# Function to get eligible treatments for each worker based on their Job
def get_eligible_treatments(worker_job, worker_treatments):
    return worker_treatments.get(worker_job, [])

# Create a new column in the DataFrame for eligible treatments
worker_df['Eligible_Treatments'] = worker_df['Job'].apply(lambda job: get_eligible_treatments(job, worker_treatments))

# Prepare to track worker hours
worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_last_end_time = {worker_id: pd.to_datetime(forecast_reshaped['Date'].min()) for worker_id in worker_df['Worker_Id']}

# Prepare to store the scheduled output
output_schedule = {room: [] for room in chairs_in_rooms.keys()}

# Initialize time tracking for each room
current_time = {room: pd.to_datetime(f"{forecast_reshaped['Date'].min()} {clinic_open_time}") for room in chairs_in_rooms.keys()}

# Keep track of which room to assign the next non-advanced treatment
non_advanced_room_toggle = True

# Track the current week to reset weekly hours
current_week = pd.to_datetime(forecast_reshaped['Date'].min()).isocalendar()[1]


# Sort the dates to ensure they are processed in chronological order
sorted_dates = sorted(forecast_reshaped['Date'].unique())

# print(worker_df)
        
# Iterate over each unique date in sorted order
for date in sorted_dates:
    print(f"Processing date: {date}")
    
    # Reset weekly hours if it's a new week
    week_of_year = pd.to_datetime(date).isocalendar()[1]
    if week_of_year != current_week:
        worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
        current_week = week_of_year
    
    # Reset daily hours for workers
    worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
    
    # Reset or update last end time appropriately for new day
    for worker_id in worker_last_end_time:
        worker_last_end_time[worker_id] = pd.to_datetime(date).replace(hour=0, minute=0, second=0)
    
    # Reset current time for each room to the clinic's opening time
    current_time = {room: pd.to_datetime(f"{date} {clinic_open_time}") for room in chairs_in_rooms.keys()}
    current_time['Remote'] = pd.to_datetime(f"{date} {clinic_open_time}")  # Add Remote room
    
    # Diagnostic output to ensure reset is correct
    print(f"Worker hours reset for {date}:")
    print(f"Daily hours: {worker_hours_daily}")
    print(f"Weekly hours: {worker_hours_weekly}")
    print(f"Worker last end time: {worker_last_end_time}")
    print(f"Current time for rooms: {current_time}")
    
    # Continue with the scheduling logic
    daily_treatments = forecast_reshaped[forecast_reshaped['Date'] == date]

    for index, row in daily_treatments.iterrows():
        treatment_id = row['Treatment']
        count = int(row['Count'])  # Number of sessions for this treatment

        # Use regex to extract item_number and variation_code from treatment_id
        match = re.match(r'(\d+)-([A-Za-z]+)', treatment_id)
        if match:
            item_number = match.group(1)
            variation_code = match.group(2)
        else:
            raise ValueError(f"Invalid treatment_id format: {treatment_id}")
        
        # Initialize variables
        treatment_info = None
        variation_info = None

        # Find the corresponding treatment and variation in the JSON data
        for treatment in item_numbers_data['treatment']:
            for detail in treatment['treatment_details']:
                # Check if the item number matches
                if str(detail['item_number']) == item_number:
                    for variation in detail['variation']:
                        # Check if the variation code matches
                        if variation['variation_code'] == variation_code:
                            treatment_info = detail
                            variation_info = variation
                            break
                    if treatment_info is not None:
                        break
            if treatment_info is not None:
                break

        # If the treatment or variation info is still None, raise an error
        if treatment_info is None or variation_info is None:
            raise ValueError(f"Could not find treatment or variation information for {treatment_id}")

        # Handle treatments without procedures or with None
        procedures = variation_info.get('item_procedures', None)
        if not procedures:  # This will cover both None and empty list cases
            procedures = [{"procedure": "{-}"}]  # Set procedure to "{-}"
            procedure_duration = total_duration = float(variation_info['variation_duration'])
        else:
            total_duration = float(variation_info['variation_duration'])
            procedure_duration = total_duration / len(procedures)

        # Select the worker for this treatment
        eligible_workers = [worker_id for worker_id in worker_df['Worker_Id'] 
                            if treatment_id in worker_df.loc[worker_df['Worker_Id'] == worker_id, 'Eligible_Treatments'].values[0]]
        selected_worker = None

        # Determine the room based on the chair
        if variation_info['chair'] == 0:
            room = 'Remote'  # Assign to Remote room
        else:
            room = 'Room3' if variation_info['chair'] == 3 else 'Room1' if non_advanced_room_toggle else 'Room2'
            non_advanced_room_toggle = not non_advanced_room_toggle
        
        for worker_id in eligible_workers:
            daily_available = max_working_hour_day - worker_hours_daily[worker_id]
            weekly_available = max_working_hour_week - worker_hours_weekly[worker_id]
            start_time = current_time[room]
            end_time = start_time + pd.to_timedelta(procedure_duration, unit='m')
            
            # Debugging of Proposed Worker to satisfy the demands
            print(f"Worker: {worker_id}, daily: {daily_available}, Weekly: {weekly_available}, Proposed start time: {start_time}, Proposed end time: {end_time}, Treatment: {treatment_id}, Room: {room}")

            if (daily_available >= procedure_duration and
                weekly_available >= procedure_duration and
                worker_last_end_time[worker_id] <= start_time):
                selected_worker = worker_id
                worker_last_end_time[worker_id] = end_time
                break
        
        if selected_worker is None:
            raise ValueError(f"No available worker for treatment {treatment_id} on {date}")
        

        #Labor Cost calculation each procedure
        labor_cost = total_duration * (worker_wages[selected_worker] / 60)

        # Schedule each procedure individually for the selected worker
        for procedure in procedures:
            start_time = current_time[room]
            end_time = start_time + pd.to_timedelta(procedure_duration, unit='m')

            # Schedule the treatment
            output_schedule[room].append({
                'Date': pd.to_datetime(date).date(),
                'Start_Time': start_time.strftime('%I:%M %p'),
                'Finish_Time': end_time.strftime('%I:%M %p'),
                'Item_Number': treatment_id,
                'Treatment_Names': treatment_info['item_description'],
                'Procedure': procedure['procedure'],
                'Duration': procedure_duration,
                'Worker_Id': selected_worker,
                'Chair': 'Remote' if room == 'Remote' else 'Chair3' if room == 'Room3' else 'Chair1' if treatment_id in chair1_treatments else 'Chair2',
                'LaborCost': labor_cost
            })
            
            current_time[room] = end_time

        current_time[room] += pd.to_timedelta(treatment_interval, unit='m')
        
        # Update worker hours
        worker_hours_daily[selected_worker] += total_duration
        worker_hours_weekly[selected_worker] += total_duration
        
        # Check if we exceed the clinic's operating hours for this room
        if current_time[room].time() > clinic_close_time:
            break
            
# Convert the schedule dictionary to DataFrames for each room
room_schedules = {room: pd.DataFrame(schedule) for room, schedule in output_schedule.items()}

# Add the Total_Cost column and calculate the total labor cost for each room
for room, schedule in room_schedules.items():
    total_cost = schedule['LaborCost'].sum()
    # Add a new 'Total_Cost' column and insert the total cost in the first row
    total_cost_series = pd.Series([total_cost] + [None] * (len(schedule) - 1))
    schedule['Total_Cost'] = total_cost_series  # Add it as the last column

# Save the output to an Excel file with separate sheets for each room
output_file_path = './clinic_scheduleV2_3.xlsx'
with pd.ExcelWriter(output_file_path) as writer:
    for room, schedule in room_schedules.items():
        schedule.to_excel(writer, sheet_name=room, index=False)

Processing date: 2024-09-03 21:01:48.797914
Worker hours reset for 2024-09-03 21:01:48.797914:
Daily hours: {'N1': 0, 'N2': 0, 'OHT1': 0, 'OHT2': 0, 'OHT3': 0, 'OHT4': 0, 'OHT5': 0, 'OHT6': 0, 'OHT7': 0, 'D1N1': 0, 'D1N2': 0, 'D1N3': 0, 'D1N4': 0, 'D2N1': 0, 'D2N2': 0, 'D2N3': 0, 'D3N1': 0, 'D3N2': 0, 'D3N3': 0}
Weekly hours: {'N1': 0, 'N2': 0, 'OHT1': 0, 'OHT2': 0, 'OHT3': 0, 'OHT4': 0, 'OHT5': 0, 'OHT6': 0, 'OHT7': 0, 'D1N1': 0, 'D1N2': 0, 'D1N3': 0, 'D1N4': 0, 'D2N1': 0, 'D2N2': 0, 'D2N3': 0, 'D3N1': 0, 'D3N2': 0, 'D3N3': 0}
Worker last end time: {'N1': Timestamp('2024-09-03 00:00:00.797914'), 'N2': Timestamp('2024-09-03 00:00:00.797914'), 'OHT1': Timestamp('2024-09-03 00:00:00.797914'), 'OHT2': Timestamp('2024-09-03 00:00:00.797914'), 'OHT3': Timestamp('2024-09-03 00:00:00.797914'), 'OHT4': Timestamp('2024-09-03 00:00:00.797914'), 'OHT5': Timestamp('2024-09-03 00:00:00.797914'), 'OHT6': Timestamp('2024-09-03 00:00:00.797914'), 'OHT7': Timestamp('2024-09-03 00:00:00.797914'), 'D1N1'

## Run Sheet Optimization V2.2 Australian data

In [23]:
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

#Load data
TaskClassification_directory_v2 = "./Data/TaskClassificationV2-2.xlsx"
scenario_df_v2 = pd.read_csv("./Data/Real/Input_for_Preprocess_2.csv")
constraints_df = pd.read_excel(TaskClassification_directory_v2, sheet_name='Constraints')
worker_df = pd.read_excel(TaskClassification_directory_v2, sheet_name='Worker')
with open("./treatment_precedence_au.json", "r") as file:
    item_numbers_data = json.load(file)

# Define the clinic conditions based on constraints
num_rooms = int(constraints_df.loc[constraints_df['Constraints'] == 'Room_num', 'Value'].values[0])
treatment_interval = int(constraints_df.loc[constraints_df['Constraints'] == 'Treatment_Interval', 'Value'].values[0])
clinic_open_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Open', 'Value'].values[0]
clinic_close_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Close', 'Value'].values[0]
max_working_hour_day = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Day', 'Value'].values[0]) * 60  # Convert hours to minutes
max_working_hour_week = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Week', 'Value'].values[0]) * 60  # Convert hours to minutes

#Preprocess
scenario_df_v2 = scenario_df_v2.iloc[:, 1:]


# Room Configurations
chairs_in_rooms = {
    'Room1': ['Chair1', 'Chair2'],
    'Room2': ['Chair1', 'Chair2'],
    'Room3': ['Chair3']
}

# Determine which worker do which treatment
def get_worker_treatments(worker_code, json_data):
    worker_treatments_list = []

    for treatment in json_data['treatment']:
        for detail in treatment['treatment_details']:
            if worker_code in detail.get('worker_responsibility_ind', []):
                worker_treatments_list.append(str(detail['item_number']))

    return worker_treatments_list

def get_item_numbers(json_data):
    item_numbers = []

    for treatment in json_data['treatment']:
        for detail in treatment['treatment_details']:
            item_numbers.append(str(detail['item_number']))

    return item_numbers

def get_chair_treatments(json_data, chair_input):
    chair_treatments = []

    for treatment in json_data['treatment']:
        for detail in treatment['treatment_details']:
            if detail['chair'] == chair_input:
                chair_treatments.append(str(detail['item_number']))

    return chair_treatments


worker_treatments = {
    "MD" : get_worker_treatments("MD", item_numbers_data),
    "OS" : get_worker_treatments("OS", item_numbers_data),
    "ORT" : get_worker_treatments("ORT", item_numbers_data),
    "GP" : get_worker_treatments("GP", item_numbers_data),
    "SPC" : get_worker_treatments("SPC", item_numbers_data),
    "PAT" : get_worker_treatments("PAT", item_numbers_data),
    "PED DNT" : get_worker_treatments("PED DNT", item_numbers_data),
    "ANT" : get_worker_treatments("ANT", item_numbers_data),
    "All SPC" : get_worker_treatments("All SPC", item_numbers_data),
    "LAB" : get_worker_treatments("LAB", item_numbers_data),
    "PER" : get_worker_treatments("PER", item_numbers_data),
    "END" : get_worker_treatments("END", item_numbers_data),
    "PRS" : get_worker_treatments("PRS", item_numbers_data),
    "RAD" : get_worker_treatments("RAD", item_numbers_data)   
}

print(worker_treatments)


# Reshape the forecast_2_weeks DataFrame to have a 'Treatment' and 'Count' for each date
forecast_reshaped = scenario_df_v2.melt(id_vars=['date'], 
                                     var_name='Treatment', 
                                     value_name='Count')


# Filter out treatments not in item_numbers
item_numbers = get_item_numbers(item_numbers_data)
forecast_reshaped = forecast_reshaped[forecast_reshaped['Treatment'].isin(item_numbers)]

# Map worker types to their hourly wages and IDs
worker_wages = worker_df.set_index('Worker_Id')['Wage_Hour'].to_dict()
job_to_worker_ids = worker_df.groupby('Job')['Worker_Id'].apply(list).to_dict() #Change the "JOB" Column format later 

# Function to get eligible treatments for each worker based on their Job
def get_eligible_treatments(worker_job, worker_treatments):
    return worker_treatments.get(worker_job, [])

# Create a new column in the DataFrame for eligible treatments
worker_df['Eligible_Treatments'] = worker_df['Job'].apply(lambda job: get_eligible_treatments(job, worker_treatments))

# Prepare to track worker hours
worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_last_end_time = {worker_id: pd.to_datetime(forecast_reshaped['date'].min()) for worker_id in worker_df['Worker_Id']}

# Prepare to store the scheduled output
output_schedule = {room: [] for room in chairs_in_rooms.keys()}

# Initialize time tracking for each room
current_time = {room: pd.to_datetime(f"{forecast_reshaped['date'].min()} {clinic_open_time}") for room in chairs_in_rooms.keys()}

# Keep track of which room to assign the next non-advanced treatment
non_advanced_room_toggle = True

# Track the current week to reset weekly hours
current_week = pd.to_datetime(forecast_reshaped['date'].min()).isocalendar()[1]

# Ensure the date column is parsed as datetime objects
forecast_reshaped['date'] = pd.to_datetime(forecast_reshaped['date'], format='%m/%d/%Y')

# Sort the dates to ensure they are processed in chronological order
sorted_dates = sorted(forecast_reshaped['date'].unique())

# Iterate over each unique date in sorted order
for date in sorted_dates:
    print(f"Processing date: {date}")
    
    # Reset weekly hours if it's a new week
    week_of_year = pd.to_datetime(date).isocalendar()[1]
    if week_of_year != current_week:
        worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
        current_week = week_of_year
    
    # Reset daily hours for workers
    worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
    
    # Reset or update last end time appropriately for new day
    for worker_id in worker_last_end_time:
        worker_last_end_time[worker_id] = pd.to_datetime(date).replace(hour=0, minute=0, second=0)
    
    # Reset current time for each room to the clinic's opening time
    current_time = {room: pd.to_datetime(f"{date} {clinic_open_time}") for room in chairs_in_rooms.keys()}
    current_time['Remote'] = pd.to_datetime(f"{date} {clinic_open_time}")  # Add Remote room
    
    # Diagnostic output to ensure reset is correct
    print(f"Worker hours reset for {date}:")
    print(f"Daily hours: {worker_hours_daily}")
    print(f"Weekly hours: {worker_hours_weekly}")
    print(f"Worker last end time: {worker_last_end_time}")
    print(f"Current time for rooms: {current_time}")
    
    # Continue with the scheduling logic
    daily_treatments = forecast_reshaped[forecast_reshaped['date'] == date]

    for index, row in daily_treatments.iterrows():
        item_number = row['Treatment']
        count = int(row['Count'])  # Number of sessions for this treatment

        # Initialize variables
        treatment_info = None
        treatment_duration = None

        # Variation factor for treatment duration
        variation_factor = np.random.uniform(0.8, 1.2)  # ±20% variability

        # Find the corresponding treatment and variation in the JSON data
        for treatment in item_numbers_data['treatment']:
            for detail in treatment['treatment_details']:
                if str(detail['item_number']) == item_number:
                    treatment_info = detail
                    treatment_duration = float(detail['treatment_duration_ind'] * variation_factor)
                    break
            if treatment_info is not None:
                break

        # Chair/room assignment
        if treatment_info['chair'] == 3:
            room = 'Room3'  # Assign to Room3 for chair 3
        else:
            room = 'Room1' if non_advanced_room_toggle else 'Room2'  # Toggle between Room1 and Room2 for chair 1 or 2
            non_advanced_room_toggle = not non_advanced_room_toggle  # Alternate between Room1 and Room2

        # Select the worker for this treatment from all eligible workers
        eligible_workers = []
        for worker_id in worker_df['Worker_Id']:
            # Check if the worker is eligible for this treatment
            if item_number in worker_df.loc[worker_df['Worker_Id'] == worker_id, 'Eligible_Treatments'].values[0]:
                eligible_workers.append(worker_id)

        selected_worker = None

        # Iterate through eligible workers and find the first available one
        for worker_id in eligible_workers:
            daily_available = max_working_hour_day - worker_hours_daily[worker_id]
            weekly_available = max_working_hour_week - worker_hours_weekly[worker_id]
            start_time = current_time[room]
            end_time = start_time + pd.to_timedelta(treatment_duration, unit='m')

            # Debugging output to validate worker selection logic
            print(f"Worker: {worker_id}, daily available: {daily_available}, weekly available: {weekly_available}, start: {start_time}, end: {end_time}")

            # Check if the worker has enough daily and weekly available time and can start the treatment on time
            if daily_available >= treatment_duration and weekly_available >= treatment_duration:
                if worker_last_end_time[worker_id] <= start_time:
                    selected_worker = worker_id
                    worker_last_end_time[worker_id] = end_time
                    break  # Stop once a worker is selected

        # If no worker is found, print debug info and raise an error
        if selected_worker is None:
            print(f"Eligible workers: {eligible_workers}")
            print(f"Daily worker hours: {worker_hours_daily}")
            print(f"Weekly worker hours: {worker_hours_weekly}")
            raise ValueError(f"No available worker for treatment {item_number} on {date}")

        # Labor Cost calculation for each procedure
        labor_cost = treatment_duration * (worker_wages[selected_worker] / 60)

        # Schedule the treatment
        output_schedule[room].append({
            'Date': pd.to_datetime(date).date(),
            'Start_Time': start_time.strftime('%I:%M %p'),
            'Finish_Time': end_time.strftime('%I:%M %p'),
            'Item_Number': item_number,
            'Treatment_Names': treatment_info['treatment_type'],
            'Duration': treatment_duration,
            'Worker_Id': selected_worker,
            'Chair': treatment_info['chair'],
            'LaborCost': labor_cost
        })

        # Update the current time for the room
        current_time[room] = end_time + pd.to_timedelta(treatment_interval, unit='m')

        # Update worker hours
        worker_hours_daily[selected_worker] += treatment_duration
        worker_hours_weekly[selected_worker] += treatment_duration

        # Check if we exceed the clinic's operating hours for this room
        if current_time[room].time() > clinic_close_time:
            break

            
# Convert the schedule dictionary to DataFrames for each room
room_schedules = {room: pd.DataFrame(schedule) for room, schedule in output_schedule.items()}

# Add the Total_Cost column and calculate the total labor cost for each room
for room, schedule in room_schedules.items():
    if 'LaborCost' in schedule.columns:
        total_cost = schedule['LaborCost'].sum()
        total_cost_series = pd.Series([total_cost] + [None] * (len(schedule) - 1))
        schedule['Total_Cost'] = total_cost_series
    else:
        schedule['Total_Cost'] = pd.Series([None] * len(schedule))  # Add an empty column

# Save the output to an Excel file with separate sheets for each room
output_file_path = './clinic_scheduleV2-2.xlsx'
with pd.ExcelWriter(output_file_path) as writer:
    for room, schedule in room_schedules.items():
        schedule.to_excel(writer, sheet_name=room, index=False)






{'MD': ['928'], 'OS': ['361', '363', '314', '371', '373', '375', '376', '377', '378', '379', '384', '386', '388', '389', '391', '392', '393', '394', '395', '399', '981', '986', '987', '365', '366', '51', '59', '432', '438', '331', '332', '337', '338', '341', '343', '344', '345', '322', '323', '324', '351', '352', '353', '354', '355', '359'], 'ORT': ['862', '881', '851', '829', '831', '833', '841', '842', '843', '845', '846', '381', '382', '385', '982', '983', '984', '985', '871', '872', '873', '874', '875', '876', '877', '878', '823', '824', '825'], 'GP': ['521', '522', '523', '524', '525', '531', '532', '533', '534', '535', '941', '642', '643', '649', '651', '652', '653', '655', '656', '658', '659', '611', '613', '625', '627', '111', '113', '114', '115', '116', '119', '741', '743', '744', '745', '746', '751', '752', '753', '754', '761', '762', '763', '764', '765', '766', '768', '711', '712', '713', '714', '715', '719', '721', '722', '723', '724', '730', '731', '732', '733', '737', '11

In [24]:
# Initialize the JSON structure
json_output = {
    "payload": []
}

# Process each room's schedule and append to the JSON output
for room, schedule in room_schedules.items():
    room_schedule = []
    total_labor_cost = schedule['LaborCost'].sum() if 'LaborCost' in schedule.columns else 0
    
    # Iterate over each row in the room's schedule and convert it to the desired format
    for _, row in schedule.iterrows():
        room_schedule.append({
            "date": row['Date'].strftime('%Y-%m-%d'),
            "appointment_start": row['Start_Time'],
            "item_number": row['Item_Number'],
            "treatment_type": row['Treatment_Names'],
            "treatment_duration": row['Duration'],
            "worker_id": row['Worker_Id'],
            "chair": row['Chair'],
            "labor_cost": row['LaborCost']
        })
    
    # Append the room information and its schedule to the payload
    json_output['payload'].append({
        "room": int(room[-1]),  # Extract the room number from the room key (e.g., 'Room1' -> 1)
        "total_labor_cost": total_labor_cost,
        "schedule": room_schedule
    })

# Output the final JSON format
output_json = json.dumps(json_output, indent=4)

# Save the JSON to a file
output_json_path = './clinic_schedule_output.json'
with open(output_json_path, 'w') as json_file:
    json_file.write(output_json)

# Print JSON for preview (optional)
print(output_json)

{
    "payload": [
        {
            "room": 1,
            "total_labor_cost": 18737683.109325938,
            "schedule": [
                {
                    "date": "2024-09-11",
                    "appointment_start": "08:00 AM",
                    "item_number": "11",
                    "treatment_type": "Comprehensive oral examination",
                    "treatment_duration": 26.06605652530304,
                    "worker_id": "GP1",
                    "chair": 1,
                    "labor_cost": 19549.54239397728
                },
                {
                    "date": "2024-09-11",
                    "appointment_start": "08:31 AM",
                    "item_number": "13",
                    "treatment_type": "Oral examination \u2013 limited",
                    "treatment_duration": 30.044436676713936,
                    "worker_id": "GP1",
                    "chair": 1,
                    "labor_cost": 22533.32750753545
                },
        

## Run sheet Scheduling V3

In [12]:
import pandas as pd
import numpy as np

# Load data
TaskClassification_directory = "./Data/TaskClassification_exp.xlsx"
treatment_scenario = pd.read_csv("./Data/Real/Model_Input_Preprocess_v2.csv")
treatment_data = pd.read_excel("./Data/Real/DES - Clarity Dental.xlsx", sheet_name="Model_Input")
constraints_df = pd.read_excel(TaskClassification_directory, sheet_name='Constraints')
worker_df = pd.read_excel(TaskClassification_directory, sheet_name='Worker_v3')

# Processing data
treatment_scenario_percentage = treatment_scenario.iloc[:len(treatment_scenario) - 2, 1:]
treatment_scenario_multiplayer = treatment_scenario.iloc[-2, 1:]
treatment_scenario_code = treatment_scenario.iloc[:len(treatment_scenario) - 2, 0]

treatment_scenario = pd.concat([treatment_scenario_code, treatment_scenario_percentage.mul(treatment_scenario_multiplayer.iloc[0], axis=1)], axis=1).round()

# treatment_scenario.drop(index=treatment_scenario.index[-1], axis=0, inplace=True)

treatment_data = treatment_data.iloc[:, :9]
treatment_data.drop(index=treatment_data.index[-1], axis=0, inplace=True)

# Ensure both treatment_scenario and treatment_data have the same type for 'Code'
treatment_scenario['Code'] = treatment_scenario['Code'].astype(str)
treatment_data['Code'] = treatment_data['Code'].astype(str)

# Define the clinic conditions based on constraints
num_rooms = int(constraints_df.loc[constraints_df['Constraints'] == 'Room_num', 'Value'].values[0])
treatment_interval = int(constraints_df.loc[constraints_df['Constraints'] == 'Treatment_Interval', 'Value'].values[0])
clinic_open_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Open', 'Value'].values[0]
clinic_close_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Close', 'Value'].values[0]
max_working_hour_day = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Day', 'Value'].values[0]) * 60  # Convert hours to minutes
max_working_hour_week = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Week', 'Value'].values[0]) * 60  # Convert hours to minutes

# Prepare worker information
worker_wages = worker_df.set_index('Worker_Id')['Wage_Hour'].to_dict()
worker_efficiencies = worker_df.set_index('Worker_Id')['Efficiency'].to_dict()

# Reshape the treatment scenario
forecast_reshaped = treatment_scenario.melt(id_vars=['Code'], var_name='Date', value_name='Count')

# Filter out zero treatment days
forecast_reshaped = forecast_reshaped[forecast_reshaped['Count'] > 0]

# Scheduling process
output_schedule = {f"Room{i}": [] for i in range(1, num_rooms + 1)}

# Initialize time tracking for each room
current_time = {room: pd.to_datetime(f"{forecast_reshaped['Date'].min()} {clinic_open_time}") for room in output_schedule.keys()}

# Worker hours tracking
worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_last_end_time = {worker_id: pd.to_datetime(forecast_reshaped['Date'].min()) for worker_id in worker_df['Worker_Id']}

# Sort the dates for proper scheduling
sorted_dates = sorted(forecast_reshaped['Date'].unique())

# Current week initialization
current_week = pd.to_datetime(sorted_dates[0]).isocalendar()[1]

# Continue scheduling
for date in sorted_dates:
    # print(f"Processing date: {date}")

    # Check if the week has changed and reset weekly hours if necessary
    week_of_year = pd.to_datetime(date).isocalendar()[1]
    if week_of_year != current_week:
        worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
        current_week = week_of_year
        # print(f"New week detected. Weekly hours reset for all workers.")

    # Reset daily working hours for workers
    worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
    # Reset the current time for each room to the clinic's opening time
    current_time = {room: pd.to_datetime(f"{date} {clinic_open_time}") for room in output_schedule.keys()}
    # print(f"Daily hours reset for all workers and time reset for all rooms.")

    # Iterate over each treatment for the day
    daily_treatments = forecast_reshaped[forecast_reshaped['Date'] == date]
    for index, row in daily_treatments.iterrows():
        treatment_code = row['Code']
        treatment_count = int(row['Count'])  # Number of sessions for this treatment

        # Ensure the treatment code exists in the treatment data
        if treatment_code not in treatment_data['Code'].values:
            print(f"Warning: Treatment code {treatment_code} not found in treatment_data.")
            continue  # Skip this treatment if the code is not found

        treatment_duration = treatment_data.loc[treatment_data['Code'] == treatment_code, 'Service Time'].values[0]

        # Loop over treatment count to schedule each instance of the treatment
        for _ in range(treatment_count):
            # Find the cheapest available worker
            selected_worker = None
            for worker_id in sorted(worker_df['Worker_Id'], key=lambda x: worker_wages[x]):
                daily_available = max_working_hour_day - worker_hours_daily[worker_id]
                weekly_available = max_working_hour_week - worker_hours_weekly[worker_id]
                start_time = current_time[next(iter(output_schedule))]  # Start with the first room

                # Adjust treatment duration based on worker efficiency
                worker_efficiency = worker_efficiencies[worker_id]
                adjusted_duration = round(treatment_duration - (treatment_duration * np.random.uniform(0, worker_efficiency) / 100), 1)

                # Check if the worker is available for the entire duration
                if daily_available >= adjusted_duration and weekly_available >= adjusted_duration:
                    selected_worker = worker_id
                    break

            if selected_worker is None:
                raise ValueError(f"No available worker for treatment {treatment_code} on {date}")

            # Calculate labor cost based on adjusted duration and worker wage
            labor_cost = (adjusted_duration / 60) * worker_wages[selected_worker]

            # Choose a room in a round-robin fashion (toggle between rooms)
            room = min(output_schedule.keys(), key=lambda r: current_time[r])

            # Assign start and end time for this treatment in the selected room
            start_time = current_time[room]
            end_time = start_time + pd.to_timedelta(adjusted_duration, unit='m')

            # Schedule the treatment
            output_schedule[room].append({
                'Date': pd.to_datetime(date).date(),
                'Start_Time': start_time.strftime('%I:%M %p'),
                'Finish_Time': end_time.strftime('%I:%M %p'),
                'Treatment_Names': treatment_code,
                'Duration': adjusted_duration,
                'Worker_Id': selected_worker,
                'Labor_Cost': labor_cost
            })

            # Update the current time for the room
            current_time[room] = end_time + pd.to_timedelta(treatment_interval, unit='m')

            # Update worker hours
            worker_hours_daily[selected_worker] += adjusted_duration
            worker_hours_weekly[selected_worker] += adjusted_duration

            # Check if the treatment exceeds the clinic closing time
            if current_time[room].time() > clinic_close_time:
                break


# Convert the schedule dictionary to DataFrames for each room
room_schedules = {room: pd.DataFrame(schedule) for room, schedule in output_schedule.items()}

# Save the output to an Excel file with separate sheets for each room
output_file_path = 'clinic_scheduleV3-1_2.xlsx'
with pd.ExcelWriter(output_file_path) as writer:
    for room, schedule in room_schedules.items():
        schedule.to_excel(writer, sheet_name=f'{room}', index=False)


## Run Sheet Scheduling V3.2 (input preprocessing2)

In [35]:
import pandas as pd
import numpy as np

# Load data
TaskClassification_directory = "./Data/TaskClassification_exp.xlsx"
treatment_scenario = pd.read_csv("./Data/Real/Input_for_Preprocess_2.csv")
treatment_data = pd.read_excel("./Data/Real/DES - Clarity Dental.xlsx", sheet_name="Model_Input")
constraints_df = pd.read_excel(TaskClassification_directory, sheet_name='Constraints')
worker_df = pd.read_excel(TaskClassification_directory, sheet_name='Worker_v3')

treatment_data = treatment_data.iloc[:, :9]
treatment_data.drop(index=treatment_data.index[-1], axis=0, inplace=True)

# Ensure both treatment_scenario and treatment_data have the same type for 'Code'
treatment_data['Code'] = treatment_data['Code'].astype(str)
treatment_scenario.columns = treatment_scenario.columns.astype(str)

# Define the clinic conditions based on constraints
num_rooms = int(constraints_df.loc[constraints_df['Constraints'] == 'Room_num', 'Value'].values[0])
treatment_interval = int(constraints_df.loc[constraints_df['Constraints'] == 'Treatment_Interval', 'Value'].values[0])
clinic_open_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Open', 'Value'].values[0]
clinic_close_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Close', 'Value'].values[0]
max_working_hour_day = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Day', 'Value'].values[0]) * 60  # Convert hours to minutes
max_working_hour_week = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Week', 'Value'].values[0]) * 60  # Convert hours to minutes

# Prepare worker information
worker_wages = worker_df.set_index('Worker_Id')['Wage_Hour'].to_dict()
worker_efficiencies = worker_df.set_index('Worker_Id')['Efficiency'].to_dict()

## Reshape the forecast_2_weeks DataFrame to have a 'Treatment' and 'Count' for each date
forecast_reshaped = treatment_scenario.melt(id_vars=['date'], 
                                     var_name='Code', 
                                     value_name='Count')

# Filter out treatments with a count of 0
forecast_reshaped = forecast_reshaped[forecast_reshaped['Count'] > 0]


# Scheduling process
output_schedule = {f"Room{i}": [] for i in range(1, num_rooms + 1)}

# Initialize time tracking for each room
current_time = {room: pd.to_datetime(f"{forecast_reshaped['date'].min()} {clinic_open_time}") for room in output_schedule.keys()}

# Worker hours tracking
worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_last_end_time = {worker_id: pd.to_datetime(forecast_reshaped['date'].min()) for worker_id in worker_df['Worker_Id']}

# Sort the dates for proper scheduling
forecast_reshaped['date'] = pd.to_datetime(forecast_reshaped['date'])
sorted_dates = sorted(forecast_reshaped['date'].unique())

print(sorted_dates)

# Current week initialization
current_week = pd.to_datetime(sorted_dates[0]).isocalendar()[1]

# Continue scheduling
for date in sorted_dates:
    # print(f"Processing date: {date}")

    # Check if the week has changed and reset weekly hours if necessary
    week_of_year = pd.to_datetime(date).isocalendar()[1]
    if week_of_year != current_week:
        worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
        current_week = week_of_year
        # print(f"New week detected. Weekly hours reset for all workers.")

    # Reset daily working hours for workers
    worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
    # Reset the current time for each room to the clinic's opening time
    current_time = {room: pd.to_datetime(f"{date} {clinic_open_time}") for room in output_schedule.keys()}
    # print(f"Daily hours reset for all workers and time reset for all rooms.")

    # Iterate over each treatment for the day
    daily_treatments = forecast_reshaped[forecast_reshaped['date'] == date]
    for index, row in daily_treatments.iterrows():
        treatment_code = row['Code']
        treatment_count = int(row['Count'])  # Number of sessions for this treatment

        # Ensure the treatment code exists in the treatment data
        if treatment_code not in treatment_data['Code'].values:
            print(f"Warning: Treatment code {treatment_code} not found in treatment_data.")
            continue  # Skip this treatment if the code is not found

        treatment_duration = treatment_data.loc[treatment_data['Code'] == treatment_code, 'Service Time'].values[0]

        # Loop over treatment count to schedule each instance of the treatment
        for _ in range(treatment_count):
            # Find the cheapest available worker
            selected_worker = None
            for worker_id in sorted(worker_df['Worker_Id'], key=lambda x: worker_wages[x]):
                daily_available = max_working_hour_day - worker_hours_daily[worker_id]
                weekly_available = max_working_hour_week - worker_hours_weekly[worker_id]
                start_time = current_time[next(iter(output_schedule))]  # Start with the first room

                # Adjust treatment duration based on worker efficiency
                worker_efficiency = worker_efficiencies[worker_id]
                adjusted_duration = round(treatment_duration - (treatment_duration * np.random.uniform(0, worker_efficiency) / 100), 1)

                # Check if the worker is available for the entire duration
                if daily_available >= adjusted_duration and weekly_available >= adjusted_duration:
                    selected_worker = worker_id
                    break

            if selected_worker is None:
                raise ValueError(f"No available worker for treatment {treatment_code} on {date}")

            # Calculate labor cost based on adjusted duration and worker wage
            labor_cost = (adjusted_duration / 60) * worker_wages[selected_worker]

            # Choose a room in a round-robin fashion (toggle between rooms)
            room = min(output_schedule.keys(), key=lambda r: current_time[r])

            # Assign start and end time for this treatment in the selected room
            start_time = current_time[room]
            end_time = start_time + pd.to_timedelta(adjusted_duration, unit='m')

            # Schedule the treatment
            output_schedule[room].append({
                'Date': pd.to_datetime(date).date(),
                'Start_Time': start_time.strftime('%I:%M %p'),
                'Finish_Time': end_time.strftime('%I:%M %p'),
                'Treatment_Code': treatment_code,
                'Duration': adjusted_duration,
                'Worker_Id': selected_worker,
                'Labor_Cost': labor_cost
            })

            # Update the current time for the room
            current_time[room] = end_time + pd.to_timedelta(treatment_interval, unit='m')

            # Update worker hours
            worker_hours_daily[selected_worker] += adjusted_duration
            worker_hours_weekly[selected_worker] += adjusted_duration

            # Check if the treatment exceeds the clinic closing time
            if current_time[room].time() > clinic_close_time:
                break


# Convert the schedule dictionary to DataFrames for each room
room_schedules = {room: pd.DataFrame(schedule) for room, schedule in output_schedule.items()}

# Save the output to an Excel file with separate sheets for each room
output_file_path = 'clinic_schedulev3-2.xlsx'
with pd.ExcelWriter(output_file_path) as writer:
    for room, schedule in room_schedules.items():
        schedule.to_excel(writer, sheet_name=f'{room}', index=False)


[Timestamp('2024-09-11 00:00:00'), Timestamp('2024-09-12 00:00:00'), Timestamp('2024-09-13 00:00:00'), Timestamp('2024-09-14 00:00:00'), Timestamp('2024-09-15 00:00:00'), Timestamp('2024-09-16 00:00:00'), Timestamp('2024-09-17 00:00:00'), Timestamp('2024-09-18 00:00:00'), Timestamp('2024-09-19 00:00:00'), Timestamp('2024-09-20 00:00:00'), Timestamp('2024-09-21 00:00:00'), Timestamp('2024-09-22 00:00:00'), Timestamp('2024-09-23 00:00:00'), Timestamp('2024-09-24 00:00:00'), Timestamp('2024-09-25 00:00:00'), Timestamp('2024-09-26 00:00:00'), Timestamp('2024-09-27 00:00:00'), Timestamp('2024-09-28 00:00:00'), Timestamp('2024-09-29 00:00:00'), Timestamp('2024-09-30 00:00:00'), Timestamp('2024-10-01 00:00:00'), Timestamp('2024-10-02 00:00:00'), Timestamp('2024-10-03 00:00:00'), Timestamp('2024-10-04 00:00:00'), Timestamp('2024-10-05 00:00:00'), Timestamp('2024-10-06 00:00:00'), Timestamp('2024-10-07 00:00:00'), Timestamp('2024-10-08 00:00:00'), Timestamp('2024-10-09 00:00:00'), Timestamp('20

## Run sheet scheduling V3.3 (Put every room information in one sheet)

In [1]:
import pandas as pd
import numpy as np

# Load data
TaskClassification_directory = "./Data/TaskClassification_exp.xlsx"
treatment_scenario = pd.read_csv("./Data/Real/Input_for_Preprocess_2.csv")
treatment_data = pd.read_excel("./Data/Real/DES - Clarity Dental.xlsx", sheet_name="Model_Input")
constraints_df = pd.read_excel(TaskClassification_directory, sheet_name='Constraints')
worker_df = pd.read_excel(TaskClassification_directory, sheet_name='Worker_v3')

treatment_data = treatment_data.iloc[:, :9]
treatment_data.drop(index=treatment_data.index[-1], axis=0, inplace=True)

# Ensure both treatment_scenario and treatment_data have the same type for 'Code'
treatment_data['Code'] = treatment_data['Code'].astype(str)
treatment_scenario.columns = treatment_scenario.columns.astype(str)

# Define the clinic conditions based on constraints
num_rooms = int(constraints_df.loc[constraints_df['Constraints'] == 'Room_num', 'Value'].values[0])
treatment_interval = int(constraints_df.loc[constraints_df['Constraints'] == 'Treatment_Interval', 'Value'].values[0])
clinic_open_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Open', 'Value'].values[0]
clinic_close_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Close', 'Value'].values[0]
max_working_hour_day = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Day', 'Value'].values[0]) * 60  # Convert hours to minutes
max_working_hour_week = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Week', 'Value'].values[0]) * 60  # Convert hours to minutes

# Prepare worker information
worker_wages = worker_df.set_index('Worker_Id')['Wage_Hour'].to_dict()
worker_efficiencies = worker_df.set_index('Worker_Id')['Efficiency'].to_dict()

# Reshape the forecast DataFrame to have a 'Treatment' and 'Count' for each date
forecast_reshaped = treatment_scenario.melt(id_vars=['date'], 
                                            var_name='Code', 
                                            value_name='Count')

# Filter out treatments with a count of 0
forecast_reshaped = forecast_reshaped[forecast_reshaped['Count'] > 0]

# Scheduling process
output_schedule = []

# Initialize time tracking for each room
current_time = {f"Room{i}": pd.to_datetime(f"{forecast_reshaped['date'].min()} {clinic_open_time}") for i in range(1, num_rooms + 1)}

# Worker hours tracking
worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_last_end_time = {worker_id: pd.to_datetime(forecast_reshaped['date'].min()) for worker_id in worker_df['Worker_Id']}

# Sort the dates for proper scheduling
forecast_reshaped['date'] = pd.to_datetime(forecast_reshaped['date'])
sorted_dates = sorted(forecast_reshaped['date'].unique())

# Current week initialization
current_week = pd.to_datetime(sorted_dates[0]).isocalendar()[1]

# Continue scheduling
for date in sorted_dates:

    # Check if the week has changed and reset weekly hours if necessary
    week_of_year = pd.to_datetime(date).isocalendar()[1]
    if week_of_year != current_week:
        worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
        current_week = week_of_year

    # Reset daily working hours for workers
    worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
    # Reset the current time for each room to the clinic's opening time
    current_time = {f"Room{i}": pd.to_datetime(f"{date} {clinic_open_time}") for i in range(1, num_rooms + 1)}

    # Iterate over each treatment for the day
    daily_treatments = forecast_reshaped[forecast_reshaped['date'] == date]
    for index, row in daily_treatments.iterrows():
        treatment_code = row['Code']
        treatment_count = int(row['Count'])  # Number of sessions for this treatment

        # Ensure the treatment code exists in the treatment data
        if treatment_code not in treatment_data['Code'].values:
            print(f"Warning: Treatment code {treatment_code} not found in treatment_data.")
            continue

        treatment_duration = treatment_data.loc[treatment_data['Code'] == treatment_code, 'Service Time'].values[0]

        # Loop over treatment count to schedule each instance of the treatment
        for _ in range(treatment_count):
            # Find the cheapest available worker
            selected_worker = None
            for worker_id in sorted(worker_df['Worker_Id'], key=lambda x: worker_wages[x]):
                daily_available = max_working_hour_day - worker_hours_daily[worker_id]
                weekly_available = max_working_hour_week - worker_hours_weekly[worker_id]

                # Adjust treatment duration based on worker efficiency
                worker_efficiency = worker_efficiencies[worker_id]
                adjusted_duration = round(treatment_duration - (treatment_duration * np.random.uniform(0, worker_efficiency) / 100), 1)

                # Check if the worker is available for the entire duration
                if daily_available >= adjusted_duration and weekly_available >= adjusted_duration:
                    selected_worker = worker_id
                    break

            if selected_worker is None:
                raise ValueError(f"No available worker for treatment {treatment_code} on {date}")

            # Calculate labor cost based on adjusted duration and worker wage
            labor_cost = (adjusted_duration / 60) * worker_wages[selected_worker]

            # Assign the treatment to the next available room
            room = min(current_time.keys(), key=lambda r: current_time[r])

            # Assign start and end time for this treatment in the selected room
            start_time = current_time[room]
            end_time = start_time + pd.to_timedelta(adjusted_duration, unit='m')

            # Schedule the treatment
            output_schedule.append({
                'Date': pd.to_datetime(date).date(),
                'Start_Time': start_time.strftime('%I:%M %p'),
                'Finish_Time': end_time.strftime('%I:%M %p'),
                'Treatment_Code': treatment_code,
                'Duration': adjusted_duration,
                'Worker_Id': selected_worker,
                'Labor_Cost': labor_cost,
                'Room_number': room
            })

            # Update the current time for the room
            current_time[room] = end_time + pd.to_timedelta(treatment_interval, unit='m')

            # Update worker hours
            worker_hours_daily[selected_worker] += adjusted_duration
            worker_hours_weekly[selected_worker] += adjusted_duration

            # Check if the treatment exceeds the clinic closing time
            if current_time[room].time() > clinic_close_time:
                break

# Convert the schedule to a DataFrame and save it to a single sheet
output_schedule_df = pd.DataFrame(output_schedule)

# Save the output to an Excel file in a single sheet
output_file_path = 'clinic_scheduleV3-3.xlsx'
output_schedule_df.to_excel(output_file_path, index=False, sheet_name='Schedule')

## Experiment

In [60]:
import json
import re

with open("./treatment_precedence.json", "r") as file:
    item_numbers_data = json.load(file)

tretment_id_example = "1001-A"
treatment_id_procedures = [] 

# Use regex to extract item_number and variation_code from treatment_id
match = re.match(r'(\d+)-([A-Za-z]+)', tretment_id_example)
if match:
    item_number = match.group(1)
    variation_code = match.group(2)
    print(f"Item number: {item_number}, Variation code: {variation_code}")
else:
    raise ValueError(f"Invalid treatment_id format: {treatment_id}")


for treatment in item_numbers_data['treatment']:
    for detail in treatment['treatment_details']:
        if detail['item_number'] == int(item_number):
            for variation in detail['variation']:
                if variation['variation_code'] == variation_code:
                    for procedures in variation["item_procedures"]:
                        treatment_id_procedures.append(procedures['procedure'])


# Determine which worker do which treatment
def get_worker_treatments(worker_code, json_data):
    worker_treatments_list = []

    for treatment in json_data['treatment']:
        for detail in treatment['treatment_details']:
            for variation in detail['variation']:
                if variation['variation_worker_responsibility'] == worker_code:
                    worker_treatments_list.append(str(detail['item_number']) + "-" + variation['variation_code'])

    return worker_treatments_list

worker_treatments = {
    "OHT" : get_worker_treatments("OHT", item_numbers_data),
    "LV1" : get_worker_treatments("LV1", item_numbers_data),
    "LV2" : get_worker_treatments("LV2", item_numbers_data),
    "LV3" : get_worker_treatments("LV3", item_numbers_data)
}

print(worker_treatments)


# Extract all unique treatment codes
unique_treatment_codes = set()

for treatments in worker_treatments.values():
    unique_treatment_codes.update(treatments)

# Convert the set to a list if needed
unique_treatment_codes_list = list(unique_treatment_codes)

# Function to sort treatment codes
def sort_treatment_code(treatment_code):
    # Split the treatment code into numeric and alphabetic parts
    match = re.match(r"(\d+)-([A-Z]+)", treatment_code)
    if match:
        number_part = int(match.group(1))  # Convert the numeric part to an integer
        letter_part = match.group(2)       # Get the alphabetic part
        return (number_part, letter_part)
    return (treatment_code, "")

# Sort the treatment codes
unique_treatment_codes_list.sort(key=sort_treatment_code)

print(unique_treatment_codes_list)

Item number: 1001, Variation code: A
{'OHT': ['1001-A', '1001-C', '1002-A', '1003-A', '1003-B', '1004-A', '1004-B', '1004-C', '1004-E', '1005-A', '1005-B', '1005-C', '3002-A', '3002-B', '3003-A', '3003-B', '3003-C', '3004-A', '3005-A', '4007-A', '4008-A', '4009-A'], 'LV1': ['1001-B', '1004-D', '3014-A'], 'LV2': ['3005-B', '3015-A', '4001-A', '4002-A', '4003-A', '4005-A', '4006-A', '5007-A', '5007-B', '5008-A', '5009-A'], 'LV3': ['3005-C', '3005-D', '3006-A', '3007-A', '3008-A', '3009-A', '3010-A', '3011-A', '3012-A', '3013-A', '4004-A', '5001-A', '5002-A', '5003-A', '5004-A', '5005-A', '5006-A']}
['1001-A', '1001-B', '1001-C', '1002-A', '1003-A', '1003-B', '1004-A', '1004-B', '1004-C', '1004-D', '1004-E', '1005-A', '1005-B', '1005-C', '3002-A', '3002-B', '3003-A', '3003-B', '3003-C', '3004-A', '3005-A', '3005-B', '3005-C', '3005-D', '3006-A', '3007-A', '3008-A', '3009-A', '3010-A', '3011-A', '3012-A', '3013-A', '3014-A', '3015-A', '4001-A', '4002-A', '4003-A', '4004-A', '4005-A', '4006

In [44]:
import pandas as pd

treatment_demand = pd.read_csv("./Data/Real/Model_Input_Preprocess_v2.csv")

treatment_demand_percentage = treatment_demand.iloc[:len(treatment_demand) - 2, 1:]
treatment_demand_multiplayer = treatment_demand.iloc[-2, 1:]
treatment_demand_code = treatment_demand.iloc[:len(treatment_demand) - 2, 0]

treatment_demand =pd.concat([treatment_demand_code, treatment_demand_percentage.mul(treatment_demand_multiplayer.iloc[0], axis=1)], axis=1).round()

print(treatment_demand)

              Code  2024-09-11  2024-09-12  2024-09-13  2024-09-14  \
0               11         1.0         0.0         1.0         1.0   
1               12         2.0         2.0         2.0         1.0   
2               13         0.0         1.0         0.0         1.0   
3               14         0.0         0.0         0.0         0.0   
4               22         9.0        11.0         9.0         7.0   
..             ...         ...         ...         ...         ...   
160          88522         0.0         0.0         0.0         0.0   
161  Dr Invis Cons         0.0         0.0         0.0         0.0   
162            FIC         0.0         0.0         0.0         0.0   
163            281         0.0         0.0         0.0         0.0   
164            618         0.0         0.0         0.0         0.0   

     2024-09-15  2024-09-16  2024-09-17  2024-09-18  2024-09-19  ...  \
0           2.0         0.0         2.0         0.0         1.0  ...   
1           4.0

In [4]:
import pandas as pd

df = pd.read_excel("./Data/Real/Item_Number_au.xlsx", sheet_name="Price point Item Number (INA)")

column_name = ["Treatment Plan", "No", "Treatment Type", "Unit of Measurement", "Designation", "Duration_au", "Duration_ind", "Labour_au", "Labour_ind"]

df1 = df.iloc[5:-1, 1]
df2 = df.iloc[5:-1, 6:14]

df_concat = pd.concat([df1, df2], axis=1)

df_processed = pd.DataFrame(df_concat.values, columns=column_name)

df_processed.to_excel("./Data/Real/Item_Number_au_processed.xlsx", sheet_name="Main")


['GP' 'GP & RAD' 'RAD' 'LAB' 'OS' 'PAT' 'LAB & PAT' 'GP & all SPC' 'SPC'
 'END' 'PED DNT' 'PRS' 'PER' 'GP & PER' 'GP & OS' 'ORT' 'PER & OS'
 'GP & END' 'END & OS' 'GP & LAB' 'GP&LAB' 'END & LAB' 'PRS & END'
 'GP & END & PRS' 'GP & PRS' 'PRS & PER' 'GP & PRS & LAB' 'PRS & LAB'
 'GP &LAB' 'PRS &LAB' 'GP & PRS &LAB' 'GP & ORT' 'GP & SPC' 'GP & All SPC'
 'MD' 'ANT' 'PER & PRS' 'ORT & PRS' 'ORT &PRS' 'PER & PRS & OS'
 'PRS & PER & OS']


c:\Users\radik\miniconda3\envs\method\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [7]:
import pandas as pd
import json
import re

# Load the Excel file
file_path = './Data/Real/Item_Number_au_processed_V2.xlsx'
df = pd.read_excel(file_path)

duration_ind= df['Duration_ind'].isnull().sum()
labour_ind = df['Labour_ind'].isnull().sum()

print(f'Check if Duration_ind null : {duration_ind}')
print(f'Check if Labour_ind null : {labour_ind}')

# Fill missing 'Treatment Plan' values forward
df['Treatment Plan'].fillna(method='ffill', inplace=True)

# Function to split and clean worker responsibility string
def split_worker_responsibility(value):
    if pd.notnull(value):
        # Split the string by '&' and remove extra spaces
        return [part.strip() for part in re.split(r'&', value)]
    return None

# Create a dictionary structure following the provided template
treatments = []

# Grouping by 'Treatment Plan' to collect treatment details for each plan
for treatment_name, group in df.groupby('Treatment Plan'):
    treatment_details = []
    
    # For each treatment within the treatment plan
    for _, row in group.iterrows():
        treatment_details.append({
            "item_number": int(row["No"]) if pd.notnull(row["No"]) else None,
            "treatment_type": row["Treatment Type"] if pd.notnull(row["Treatment Type"]) else None,
            "measurement_unit": row["Unit of Measurement"] if pd.notnull(row["Unit of Measurement"]) else None,
            "designation": row["Designation"] if pd.notnull(row["Designation"]) else None,
            "chair": row['Chair'] if pd.notnull(row['Chair']) else None,
            "treatment_duration_au": int(row["Duration_au"]) if pd.notnull(row["Duration_au"]) else None,
            "treatment_duration_ind": int(row["Duration_ind"]) if pd.notnull(row["Duration_ind"]) else None,
            "worker_responsibility_au": split_worker_responsibility(row["Labour_au"]),
            "worker_responsibility_ind": split_worker_responsibility(row["Labour_ind"])
        })
    
    treatments.append({
        "treatment_name": treatment_name,
        "treatment_details": treatment_details
    })

# Create the final JSON structure
final_json = {
    "treatment": treatments
}

# Save the generated JSON structure to a file
json_file_path = './treatment_precedence_au.json'
with open(json_file_path, 'w') as json_file:
    json.dump(final_json, json_file, indent=4)

print(f"JSON file saved at {json_file_path}")


Check if Duration_ind null : 0
Check if Labour_ind null : 0
JSON file saved at ./treatment_precedence_au.json


In [16]:
import json
from collections import Counter

# Load the JSON data from the file
with open('./treatment_precedence_au.json', 'r') as file:
    data = json.load(file)

# Initialize an empty set to store unique values
unique_worker_responsibility_ind = set()
worker_responsibility_ind_list = []

# Traverse through the JSON structure
for treatment in data.get('treatment', []):
    for detail in treatment.get('treatment_details', []):
        responsibilities = detail.get('worker_responsibility_ind', [])
        # Add all unique responsibilities to the set
        unique_worker_responsibility_ind.update(responsibilities)
        worker_responsibility_ind_list.extend(responsibilities)

# Convert the set back to a list if needed
unique_worker_responsibility_ind_list = list(unique_worker_responsibility_ind)

# Output the unique values
print(unique_worker_responsibility_ind_list)

# Use Counter to count occurrences of each unique responsibility
worker_responsibility_ind_count = Counter(worker_responsibility_ind_list)

# Output the count of each unique worker_responsibility_ind
for responsibility, count in worker_responsibility_ind_count.items():
    print(f"{responsibility}: {count}")

['SPC', 'END', 'PER', 'PED DNT', 'PRS', 'ORT', 'RAD', 'GP', 'PAT', 'All SPC', 'OS', 'ANT', 'MD', 'LAB']
{'8', '9', '0', '1', '5', '3', '7', '6', '4', '2'}
GP: 163
END: 33
ANT: 6
ORT: 29
PRS: 97
LAB: 54
OS: 46
All SPC: 4
MD: 1
SPC: 6
PER: 36
PAT: 4
RAD: 15
PED DNT: 4
